# Loading a TorchScript Model in C++

For production scenarios, C++ is very often the language of choice, even if only to bind it into another language like Java, Rust or Go. The following paragraphs will outline the path PyTorch provides to go from an existing Python model to a serialized representation that can be loaded and executed purely from C++, with no dependency on Python.

# Step 1: Converting Your PyTorch Model to Torch Script

A PyTorch model’s journey from Python to C++ is enabled by Torch Script, a representation of a PyTorch model that can be understood, compiled and serialized by the Torch Script compiler. If you are starting out from an existing PyTorch model written in the vanilla “eager” API, you must first convert your model to Torch Script. In the most common cases, discussed below, this requires only little effort. 

If you already have a Torch Script module, you can skip to the next section of this tutorial.

There exist two ways of converting a PyTorch model to Torch Script. 

The first is known as tracing, a mechanism in which the structure of the model is captured by evaluating it once using example inputs, and recording the flow of those inputs through the model. 

This is suitable for models that make limited use of control flow. 

The second approach is to add explicit annotations to your model that inform the Torch Script compiler that it may directly parse and compile your model code, subject to the constraints imposed by the Torch Script language.

# Tracing and getting a TorchScript Model

To convert a PyTorch model to Torch Script via tracing, you must pass an instance of your model along with an example input to the torch.jit.trace function. 

This will produce a torch.jit.ScriptModule object with the trace of your model evaluation embedded in the module’s forward method:

In [0]:
import torch
import torchvision


In [0]:
model = torchvision.models.resnet18()

In [0]:
# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 3, 224, 224)

In [0]:
# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

In [0]:
print(traced_script_module)

# Converting to Torch Script via Annotation (Scripting)

Under certain circumstances, such as if your model employs particular forms of control flow, you may want to write your model in Torch Script directly and annotate your model accordingly. For example, say you have the following vanilla Pytorch model:

In [0]:
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

Because the forward method of this module uses control flow that is dependent on the input, it is not suitable for tracing. 

Instead, we can convert it to a ScriptModule. In order to convert the module to the ScriptModule, one needs to compile the module with torch.jit.script as follows:

In [0]:
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

In [0]:
my_module = MyModule(10,20)
sm = torch.jit.script(my_module)

# Step 2: Serializing Your Script Module to a File

Once you have a ScriptModule in your hands, either from tracing or annotating a PyTorch model, you are ready to serialize it to a file. 

Later on, you’ll be able to load the module from this file in C++ and execute it without any dependency on Python. 

Say we want to serialize the ResNet18 model shown earlier in the tracing example. To perform this serialization, simply call save on the module and pass it a filename:

In [0]:
traced_script_module.save("traced_resnet_model.pt")

This will produce a traced_resnet_model.pt file in your working directory. If you also would like to serialize my_module, call my_module.save("my_module_model.pt") 

We have now officially left the realm of Python and are ready to cross over to the sphere of C++.

# Step 3: Loading Your Script Module in C++



To load your serialized PyTorch model in C++, your application must depend on the PyTorch C++ API – also known as LibTorch. 

The LibTorch distribution encompasses a collection of shared libraries, header files and CMake build configuration files. 

While CMake is not a requirement for depending on LibTorch, it is the recommended approach and will be well supported into the future. 

For this tutorial, we will be building a minimal C++ application using CMake and LibTorch that simply loads and executes a serialized PyTorch model.